In [8]:
# This is an example of how to do stacking with H2O in Python


import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from sklearn import metrics  #will be replaced with ensemble_performance later


def source_stack_utils():
	'''
	Current location of h2o stack python utils
	'''
	import urllib
	url = "https://gist.githubusercontent.com/ledell/8ba8d064ae13169a1821faac70d2211b/raw/7d0fa741df619d1a5340e06258a91831951be8a9/stack.py"
	urllib.urlretrieve(url, "stack.py")
	

def prep_data_example():

	# Import a sample binary outcome train/test set into R
	train = h2o.import_file("http://www.stat.berkeley.edu/~ledell/data/higgs_10k.csv")
	test = h2o.import_file("http://www.stat.berkeley.edu/~ledell/data/higgs_test_5k.csv")
	y = "C1"
	x = list(train.columns)
	x.pop(0)

	family = "binomial"

	#For binary classification, response should be a factor
	train[y] = train[y].asfactor()  
	test[y] = test[y].asfactor()
	
	return x, y, train, test, family


def cvtrain_base_models(x, y, train, family):
	''' 
	Here we (5-fold) cross-validate a collection of base models
	This is an example of an ensemble of nine models:
	- 3 GBM
	- 3 DL
	- 2 RF
	- 1 GLM
	'''

	# All models must use exact same CV folds
	nfolds = 5
	fold_assignment = 'Modulo'


	# TO DO: Sync up family and distribution and un-hardcode distribution below
	gbm1 = H2OGradientBoostingEstimator(distribution='bernoulli',
	                                    ntrees=100,
	                                    max_depth=4,
	                                    learn_rate=0.1,
	                                    nfolds=nfolds,
	                                    fold_assignment=fold_assignment,
	                                    keep_cross_validation_predictions=True)

	gbm2 = H2OGradientBoostingEstimator(distribution='bernoulli',
	                                    ntrees=100,
	                                    max_depth=4,
	                                    learn_rate=0.1,
	                                    col_sample_rate=0.7,
	                                    nfolds=nfolds,
	                                    fold_assignment=fold_assignment,
	                                    keep_cross_validation_predictions=True)

	gbm3 = H2OGradientBoostingEstimator(distribution='bernoulli',
	                                    ntrees=100,
	                                    max_depth=2,
	                                    learn_rate=0.1,
	                                    nfolds=nfolds,
	                                    fold_assignment=fold_assignment,
	                                    keep_cross_validation_predictions=True)

	dl1 = H2ODeepLearningEstimator(distribution='bernoulli',
	                               activation='Rectifier', 
	                               hidden=[50,50,50],
	                               l1=1e-5, 
	                               epochs=10,
	                               nfolds=nfolds,
	                               fold_assignment=fold_assignment,
	                               keep_cross_validation_predictions=True)

	dl2 = H2ODeepLearningEstimator(distribution='bernoulli',
	                               activation='RectifierWithDropout', 
	                               hidden=[100,100,100],
	                               input_dropout_ratio=0.2, 
	                               l1=1e-5, 
	                               epochs=10,
	                               nfolds=nfolds,
	                               fold_assignment=fold_assignment,
	                               keep_cross_validation_predictions=True)

	dl3 = H2ODeepLearningEstimator(distribution='bernoulli',
	                               activation='Rectifier', 
	                               hidden=[200,200],
	                               l1=1e-6, 
	                               epochs=10,
	                               nfolds=nfolds,
	                               fold_assignment=fold_assignment,
	                               keep_cross_validation_predictions=True)

	rf1 = H2ORandomForestEstimator(#distribution='bernoulli',
	                               ntrees=300, 
	                               nfolds=nfolds,
	                               fold_assignment=fold_assignment,
	                               keep_cross_validation_predictions=True)

	rf2 = H2ORandomForestEstimator(#distribution='bernoulli',
	                               ntrees=300,
	                               sample_rate=0.7,
	                               mtries=10, 
	                               nfolds=nfolds,
	                               fold_assignment=fold_assignment,
	                               keep_cross_validation_predictions=True)

	glm1 = H2OGeneralizedLinearEstimator(family='binomial',
	                                     nfolds=nfolds,
	                                     fold_assignment=fold_assignment,
	                                     keep_cross_validation_predictions=True)

	# Edit this list of base models to make different ensembles
	models = [gbm1, gbm2, gbm3, dl1, dl2, dl3, rf1, rf2, glm1]

	for model in models:
		model.train(x=x, y=y, training_frame=train)

	return models



def main():

	h2o.init()

	# Load some example binary response data
	x, y, train, test, family = prep_data_example()
	
	# Load stacking utils
	source_stack_utils()
	from stack import make_Z, get_cvpreds, stack, metapredict

	# Cross-validation & training of base models
	# Above we train an abitrary assortment of base models
	models = cvtrain_base_models(x=x, y=y, train=train, family=family)

	# Define a NN-GLM metalearner
	metalearner = H2OGeneralizedLinearEstimator(family='binomial', non_negative=True)

	# Fit the stacked ensemble / Super Learner
	metafit = stack(models=models, 
			metalearner=metalearner,
			response_frame=train[y],
			seed=1,
			keep_levelone_data=True)

	# Generate ensemble prediction on the test set
	pred, basepred = metapredict(models=models, metafit=metafit, test_data=test)

	# TO DO: Add metafit.ensemble_performance()
	# Evaluate ensemble test performance
	preds = pred[2].as_data_frame(True)
	labels = test[y].as_data_frame(True)
	fpr, tpr, thresholds = metrics.roc_curve(labels, preds, pos_label=1)
	auc = metrics.auc(fpr, tpr)
	print str(auc) + "  " + "H2O Ensemble" 

	# Evaluate base learner test set performance (for comparison)
	for model in models:
		bperf = model.model_performance(test_data=test)
		print str(bperf.auc()) + "  " + model.model_id


In [ ]:
def prep_data_example():

	# Import a sample binary outcome train/test set into R
	train = h2o.import_file("http://www.stat.berkeley.edu/~ledell/data/higgs_10k.csv")
	test = h2o.import_file("http://www.stat.berkeley.edu/~ledell/data/higgs_test_5k.csv")
	y = "C1"
	x = list(train.columns)
	x.pop(0)

	family = "binomial"

	#For binary classification, response should be a factor
	train[y] = train[y].asfactor()  
	test[y] = test[y].asfactor()
	
	return x, y, train, test, family


In [9]:
main()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,5 mins 10 secs
H2O cluster version:,3.10.4.8
H2O cluster version age:,1 month and 19 days
H2O cluster name:,H2O_from_python_root_bto0ry
H2O cluster total nodes:,1
H2O cluster free memory:,1.579 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
deeplearning Model Build progress: |██████████████████████████████████████| 100%
deeplearning Model Build progress: |██████████████████████████████████████| 100%
deeplearning Model Build progress: |██████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%


H2OResponseError: Server error java.lang.IllegalArgumentException:
  Error: Expected a Frame but found a class water.rapids.vals.ValRow
  Request: POST /99/Rapids
    data: {u'session_id': '_sid_9d29', u'ast': '(tmp= py_73_sid_9d29 (apply (cbind py_16_sid_9d29 (cols_py prediction_GBM_model_python_1499764505931_7_cv_5 2)) 1 { row . (sumNA row True 0) }))'}


In [5]:
my_gbm.model_performance()


ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 23.9028739906
RMSE: 4.88905655424
MAE: 3.91630132575
RMSLE: 0.0753989795312
Mean Residual Deviance: 23.9028739906


In [6]:
my_rf.model_performance()


ModelMetricsRegression: drf
** Reported on train data. **

MSE: 48.5953005986
RMSE: 6.97103296496
MAE: 5.65954158432
RMSLE: 0.106521181598
Mean Residual Deviance: 48.5953005986


In [7]:
my_glm.model_performance()


ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 40.5757406849
RMSE: 6.3699090013
MAE: 5.22359545593
RMSLE: 0.0977400498009
R^2: 0.0443128952058
Mean Residual Deviance: 40.5757406849
Null degrees of freedom: 303
Residual degrees of freedom: 291
Null deviance: 12906.9703947
Residual deviance: 12335.0251682
AIC: 2016.47842004


In [10]:
import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
from h2o.grid.grid_search import H2OGridSearch
from __future__ import print_function
h2o.init()

# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")

# Identify predictors and response
x = train.columns
y = "response"
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Number of CV folds (to generate level-one data for stacking)
nfolds = 5

# There are a few ways to assemble a list of models to stack together:
# 1. Train individual models and put them in a list
# 2. Train a grid of models
# 3. Train several grids of models
# Note: All base models must have the same cross-validation folds and
# the cross-validated predicted values must be kept.


# 1. Generate a 2-model ensemble (GBM + RF)

# Train and cross-validate a GBM
my_gbm = H2OGradientBoostingEstimator(distribution="bernoulli",
                                      ntrees=10,
                                      max_depth=3,
                                      min_rows=2,
                                      learn_rate=0.2,
                                      nfolds=nfolds,
                                      fold_assignment="Modulo",
                                      keep_cross_validation_predictions=True,
                                      seed=1)
my_gbm.train(x=x, y=y, training_frame=train)


# Train and cross-validate a RF
my_rf = H2ORandomForestEstimator(ntrees=50,
                                 nfolds=nfolds,
                                 fold_assignment="Modulo",
                                 keep_cross_validation_predictions=True,
                                 seed=1)
my_rf.train(x=x, y=y, training_frame=train)


# Train a stacked ensemble using the GBM and GLM above
ensemble = H2OStackedEnsembleEstimator(model_id="my_ensemble_binomial",
                                       base_models=[my_gbm.model_id, my_rf.model_id])
ensemble.train(x=x, y=y, training_frame=train)

# Eval ensemble performance on the test data
perf_stack_test = ensemble.model_performance(test)

# Compare to base learner performance on the test set
perf_gbm_test = my_gbm.model_performance(test)
perf_rf_test = my_rf.model_performance(test)
baselearner_best_auc_test = max(perf_gbm_test.auc(), perf_rf_test.auc())
stack_auc_test = perf_stack_test.auc()
print("Best Base-learner Test AUC:  {0}".format(baselearner_best_auc_test))
print("Ensemble Test AUC:  {0}".format(stack_auc_test))

# Generate predictions on a test set (if neccessary)
pred = ensemble.predict(test)


# 2. Generate a random grid of models and stack them together

# Specify GBM hyperparameters for the grid
hyper_params = {"learn_rate": [0.01, 0.03],
                "max_depth": [3, 4, 5, 6, 9],
                "sample_rate": [0.7, 0.8, 0.9, 1.0],
                "col_sample_rate": [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]}
search_criteria = {"strategy": "RandomDiscrete", "max_models": 3, "seed": 1}

# Train the grid
grid = H2OGridSearch(model=H2OGradientBoostingEstimator(ntrees=10,
                                                        seed=1,
                                                        nfolds=nfolds,
                                                        fold_assignment="Modulo",
                                                        keep_cross_validation_predictions=True),
                     hyper_params=hyper_params,
                     search_criteria=search_criteria,
                     grid_id="gbm_grid_binomial")

grid.train(x=x, y=y, training_frame=train)

# Train a stacked ensemble using the GBM grid
ensemble = H2OStackedEnsembleEstimator(model_id="my_ensemble_gbm_grid_binomial",
                                       base_models=grid.model_ids)
ensemble.train(x=x, y=y, training_frame=train)

# Eval ensemble performance on the test data
perf_stack_test = ensemble.model_performance(test)

# Compare to base learner performance on the test set
baselearner_best_auc_test = max([h2o.get_model(model).model_performance(test_data=test).auc() for model in grid.model_ids])
stack_auc_test = perf_stack_test.auc()
print("Best Base-learner Test AUC:  {0}".format(baselearner_best_auc_test))
print("Ensemble Test AUC:  {0}".format(stack_auc_test))

# Generate predictions on a test set (if neccessary)
pred = ensemble.predict(test)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,15 mins 37 secs
H2O cluster version:,3.10.4.8
H2O cluster version age:,1 month and 19 days
H2O cluster name:,H2O_from_python_root_bto0ry
H2O cluster total nodes:,1
H2O cluster free memory:,1.323 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
stackedensemble Model Build progress: |███████████████████████████████████| 100%
Best Base-learner Test AUC:  0.7698039263
Ensemble Test AUC:  0.773520920561
stackedensemble prediction progress: |████████████████████████████████████| 100%
gbm Grid Build progress: |████████████████████████████████████████████████| 100%
stackedensemble Model Build progress: |███████████████████████████████████| 100%
Best Base-learner Test AUC:  0.748152161235
Ensemble Test AUC:  0.752968374821
stackedensemble prediction progress: |████████████████████████████████████| 100%


In [17]:
my_rf

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1499764505931_4666


ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.205030575683
RMSE: 0.452803020842
LogLoss: 0.596943422279
Mean Per-Class Error: 0.322977785672
AUC: 0.743070388021
Gini: 0.486140776041
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.388840905704: 


,0,1,Error,Rate
0,1939.0,2766.0,0.5879,(2766.0/4705.0)
1,652.0,4643.0,0.1231,(652.0/5295.0)
Total,2591.0,7409.0,0.3418,(3418.0/10000.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3888409,0.7309509,278.0
max f2,0.1788807,0.8534045,363.0
max f0point5,0.5852782,0.6995461,177.0
max accuracy,0.4928140,0.6807,224.0
max precision,1.0,1.0,0.0
max recall,0.0000400,1.0,399.0
max specificity,1.0,1.0,0.0
max absolute_mcc,0.4928140,0.3572295,224.0
max min_per_class_accuracy,0.5261269,0.6722635,208.0
max mean_per_class_accuracy,0.4999987,0.6770222,220.0


Gains/Lift Table: Avg response rate: 52.95 %



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error
,2017-07-11 09:31:13,24.994 sec,0.0,nan,nan,nan,nan,nan
,2017-07-11 09:31:13,25.030 sec,1.0,0.6385901,13.8812897,0.5870120,1.6247862,0.4649171
,2017-07-11 09:31:13,25.065 sec,2.0,0.6136882,11.8457691,0.6004726,1.8272838,0.4701255
,2017-07-11 09:31:13,25.105 sec,3.0,0.5956079,10.3290229,0.6096115,1.8799987,0.4719436
,2017-07-11 09:31:13,25.150 sec,4.0,0.5787619,8.9658522,0.6186953,1.8870094,0.4732604
---,---,---,---,---,---,---,---,---
,2017-07-11 09:31:16,28.441 sec,33.0,0.4597213,0.6789258,0.7265641,1.8885741,0.3719
,2017-07-11 09:31:17,28.615 sec,34.0,0.4595016,0.6724601,0.7271760,1.8885741,0.3498
,2017-07-11 09:31:17,28.790 sec,35.0,0.4592923,0.6594218,0.7274484,1.8885741,0.3589
,2017-07-11 09:31:17,28.972 sec,36.0,0.4588341,0.6520514,0.7285119,1.8885741,0.3712



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
x26,7796.9423828,1.0,0.1012773
x27,4754.2626953,0.6097599,0.0617549
x28,4638.1064453,0.5948622,0.0602461
x23,3618.0004883,0.4640281,0.0469955
x25,3555.4287109,0.4560029,0.0461828
---,---,---,---
x24,1305.1898193,0.1673976,0.0169536
x9,766.6604614,0.0983283,0.0099584
x13,709.5970459,0.0910097,0.0092172
x17,657.1307373,0.0842806,0.0085357



See the whole table with table.as_data_frame()


In [16]:
my_gbm

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1499764505931_4417


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.200521707463
RMSE: 0.447796502289
LogLoss: 0.587917746409
Mean Per-Class Error: 0.296132236315
AUC: 0.773550268485
Gini: 0.547100536969
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.442397850743: 


,0,1,Error,Rate
0,2433.0,2272.0,0.4829,(2272.0/4705.0)
1,818.0,4477.0,0.1545,(818.0/5295.0)
Total,3251.0,6749.0,0.309,(3090.0/10000.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4423979,0.7434407,248.0
max f2,0.3358310,0.8588127,324.0
max f0point5,0.5424450,0.7261194,169.0
max accuracy,0.5074003,0.7042,196.0
max precision,0.7904727,0.9736842,3.0
max recall,0.1582546,1.0,394.0
max specificity,0.8039330,0.9995749,0.0
max absolute_mcc,0.5193304,0.4070389,187.0
max min_per_class_accuracy,0.5133466,0.7035069,191.0
max mean_per_class_accuracy,0.5193304,0.7038678,187.0


Gains/Lift Table: Avg response rate: 52.95 %



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error
,2017-07-11 09:30:47,1.575 sec,0.0,0.4991290,0.6914057,0.5,1.0,0.4705
,2017-07-11 09:30:47,1.586 sec,1.0,0.4882357,0.6697789,0.6891914,1.3408346,0.3927
,2017-07-11 09:30:47,1.600 sec,2.0,0.4798070,0.6531463,0.7191107,1.5450300,0.3958
,2017-07-11 09:30:47,1.616 sec,3.0,0.4736981,0.6410372,0.7246499,1.5690323,0.3933
,2017-07-11 09:30:48,1.633 sec,4.0,0.4670462,0.6278163,0.7466379,1.6838444,0.3692
,2017-07-11 09:30:48,1.653 sec,5.0,0.4629941,0.6196293,0.7508989,1.6964735,0.3404
,2017-07-11 09:30:48,1.676 sec,6.0,0.4583008,0.6099732,0.7566041,1.7335655,0.3515
,2017-07-11 09:30:48,1.704 sec,7.0,0.4555691,0.6041919,0.7580958,1.7480747,0.3512
,2017-07-11 09:30:48,1.739 sec,8.0,0.4527774,0.5983351,0.7637147,1.7544131,0.318
,2017-07-11 09:30:48,1.780 sec,9.0,0.4497426,0.5919289,0.7684922,1.7766586,0.3207


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
x26,565.3323364,1.0,0.4108595
x28,204.5039978,0.3617412,0.1486248
x27,189.2307892,0.3347249,0.1375249
x23,124.5276184,0.2202733,0.0905014
x6,121.5511169,0.2150083,0.0883382
---,---,---,---
x17,0.0,0.0,0.0
x19,0.0,0.0,0.0
x20,0.0,0.0,0.0
x21,0.0,0.0,0.0



See the whole table with table.as_data_frame()


In [18]:
print("Best Base-learner Test AUC:  {0}".format(baselearner_best_auc_test))
print("Ensemble Test AUC:  {0}".format(stack_auc_test))

Best Base-learner Test AUC:  0.748152161235
Ensemble Test AUC:  0.752968374821
